In [1]:
import pandas as pd
import datetime

Импортировал датасэт

In [2]:
df = pd.read_csv('/Users/apples/Desktop/Beeline TM/вложение 1/02_Data_test.csv', sep=';')

Проверил какое количество одного номера содержится в датасэте

In [3]:
print(len(df[df.msisdn == 158524417875]))

93


Импортировал датасэт с фактами, которые номера телефонов скорее всего принадлежат одной персоне

In [4]:
df1 = pd.read_excel('/Users/apples/Desktop/Beeline TM/вложение 1/01_Факты.xlsx', header = None)

Задаю header данным по фактам.
Далее делаю merge номеров телефонов из данных по фактам с данными о перемещении абонентов

In [5]:
df1.columns = ['Number1', 'Number2'] 
df_final = pd.merge(df, df1, how ='left', left_on = 'msisdn',right_on='Number1')
df1.columns = ['Number_1', 'Number_2']       

df_final = pd.merge(df_final, df1, how ='left', left_on = 'msisdn',right_on='Number_2')

Убираю повторяющиеся номера телефонов из df_final

In [6]:
df_final.drop(['Number1','Number_2'], axis = 1, inplace = True)

Задаю новую переменную, которая содержит те номера телефонов, которые скорее всего принадлежат 1 персоне

In [7]:
import math
def one_var(row):
     if math.isnan(row['Number2']) is False:
         return row['Number2']
     else:
         return row['Number_1']


df_final['Duplicate'] = df_final.apply(lambda row: one_var(row), axis = 1)
df_final.drop(['Number2', 'Number_1'], axis = 1, inplace = True)

Так как задача стоит определить абонентов msisdn, которые принадлежат одной персоне, логика такая:
1. Используя метод Матриц Самоподобиий (Similarity Matrix), выявить самые близкие дистанции, которые могут принадлежать одной персоне
с помощью числовых преобразований. Это поспособствует определению персон, которые скорее всего имеют несколько номеров по величине допустимой дистанций. 
2. Использовать метод KNN ближайших соседей, как модель определения более точного прогнозирования абонентов.

Так как переменная cell_type не является категориальной в данном df, привожу к соответствующему виду используя LabelEncoder

In [8]:
from sklearn.preprocessing import LabelEncoder as LE
le = LE()

le.fit(df_final.cell_type)
df_final['cell_type'] = le.transform(df_final.cell_type)

Заготовка к KNN, включает зависимую и независимые переменные. 

In [9]:
X = df_final.dropna(subset = ['Duplicate'], axis = 0)
X.dropna(subset = ['imei'], axis = 0, inplace = True)
y = X['Duplicate']
X.drop('Duplicate', axis = 1, inplace = True)

/Users/apples/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/apples/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Similarity Matrix

Провожу очистку данных неопознанных значений.

In [10]:
df_final.drop('Duplicate', axis = 1, inplace = True)
df_final.dropna(subset = ['imei'], axis = 0, inplace = True)

Провожу агрегацию по абонентам, чтобы Евклид не был громоздким. Так как строк в первоначальном DataFrame rows = 174123. 
Это очень сильно влияет на производительность кода.

In [11]:
df_final_agg = df_final.groupby(['msisdn']).mean()

In [12]:
df_final_agg.head()

,lac,cid,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
msisdn,,,,,,,,,,,
158500002513,5082.000000,43113.125000,3.573050e+14,3.625000,1.369370e+12,38.133900,56.333600,2265.000000,2.000000,231.250000,246.250000
158500006927,6821.592857,27604.271429,3.548705e+14,4.571429,1.369380e+12,37.567614,55.896908,1147.771429,2.000000,216.478571,232.521429
158500010757,7669.452174,22013.260870,3.570200e+14,4.878261,1.369392e+12,37.494490,55.862238,864.356522,1.791304,144.652174,223.434783
158500013119,7505.743802,25684.512397,3.529001e+14,2.446281,1.369383e+12,37.495393,55.730598,827.173554,1.983471,171.508264,188.822314
158500019021,5011.062500,41709.312500,3.516761e+14,4.656250,1.369417e+12,37.926522,55.975425,1272.750000,2.000000,161.875000,188.125000


In [13]:
df_final_agg.reset_index(level=0, inplace=True) 

In [14]:
import pandas as pd
from scipy.spatial.distance import euclidean, pdist, squareform


def similarity_func(u, v):
    return 1/(1+euclidean(u,v))


dists = pdist(df_final_agg, similarity_func)
DF_euclid = pd.DataFrame(squareform(dists), columns=df_final_agg.msisdn, index=df_final_agg.msisdn)


Агрегирую average всех дистанций, чтобы получить неповторяющие значения msisdn в DataFrame.
Делаю такую операцию со строчками и со столбцами.

In [15]:
df_ = pd.DataFrame(DF_euclid.groupby(DF_euclid.index).mean())
df_t = df_.T
df__ = pd.DataFrame(df_t.groupby(df_t.index).mean())
DF_euclid = df__.T

Формирую список всех значений с наименьшей дистанцией. В данном случае меньше 1.5e-15.
С этим числом можно играться и как результат будут в списке разное количество номеров телефонов.

In [16]:
postup = []
for i in DF_euclid.index:
    for j in DF_euclid.columns:
        if i != j:
            if DF_euclid.loc[i, j] < 1.5e-15:
                #print(DF_euclid.loc[i, j])
                postup.append((i, j))

In [17]:
from itertools import chain
k = list(chain.from_iterable(postup))
list_ = list(set(k))      
print(len(list_))
#print(postup[0:4])

576


Разделяю данные. Для подготовки всех допустимых вариантов номеров телефонов. Задаю таких абонентов по всей видимости имеющих одинаковые номера телефона, а остальные с большей вероятности не подошли по дистанции по методу Матриц Самоподобий.
То есть теперь у нас есть персоны, которые разделены на тех у кого есть 2 номера и тех у кого только 1.

In [18]:
df_final = df_final[df_final.msisdn.isin(list_)]

In [19]:
df_final.shape

(40426, 12)

### KNN

In [20]:
from sklearn.neighbors import KNeighborsClassifier as KNN

In [21]:
estimator = KNN(n_neighbors = 1)

In [22]:
estimator.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform')

In [23]:
y_pred = estimator.predict(df_final)

In [24]:
frame = { 'msisdn_1': df_final['msisdn'].values, 'msisdn_2':y_pred }

In [25]:
result = pd.DataFrame(frame)

In [26]:
result['person_id'] = result.index

In [27]:
result.head()

,msisdn_1,msisdn_2,person_id
0,158520145943,1.585240e+11,0
1,158534117157,1.585289e+11,1
2,158509011511,1.585289e+11,2
3,158516910421,1.585009e+11,3
4,158536543525,1.585289e+11,4


In [28]:
#result.to_csv('result.csv')